# Importing and fixing data format

This notebook imports and fix the data format to an easy working flow for python code and data understanding

In [1]:
#LOADS LIBRARIES
import os
import sys
import warnings
warnings.simplefilter('ignore')
import pickle
#import dill as pickle

import numpy as np
import pandas as pd
import math

import time 
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
os.getcwd()

'C:\\Users\\willi\\Documents\\EJERCICIOS\\sales_prediction_oe\\data_science'

In [3]:
# Defines the storage directories

INPUT_DIR = os.getcwd() + '/data/'
OUTPUT_DIR = os.getcwd() + '/results/'

# Importing the data

In [4]:
#Importing the TRAIN dataset

FILE_NAME = 'dataset_train.csv'
df_input_01 = pd.read_csv(INPUT_DIR + FILE_NAME, encoding='latin-1', sep=',', dtype='str')
df_input_01['set_description'] = '1_TRAIN'

print(df_input_01.shape)
print(df_input_01)


(51843, 4)
                   date q_current Store set_description
0         1/1/2021 0:00        87     1         1_TRAIN
1         1/1/2021 0:00       145     5         1_TRAIN
2         1/1/2021 0:00        49     9         1_TRAIN
3         1/1/2021 0:30       102     1         1_TRAIN
4         1/1/2021 0:30       170     5         1_TRAIN
...                 ...       ...   ...             ...
51838  12/26/2021 23:30        65     5         1_TRAIN
51839  12/26/2021 23:30        31     9         1_TRAIN
51840   12/27/2021 0:00        55     1         1_TRAIN
51841   12/27/2021 0:00        73     5         1_TRAIN
51842   12/27/2021 0:00        25     9         1_TRAIN

[51843 rows x 4 columns]


In [5]:
#Importing the PREDICTION dataset

FILE_NAME = 'dataset_predict.csv'
df_input_02 = pd.read_csv(INPUT_DIR + FILE_NAME, encoding='latin-1', sep=',', dtype='str')
df_input_02['set_description'] = '2_PREDICTION'

print(df_input_02.shape)
print(df_input_02)

(2304, 4)
                 date q_predicted Store set_description
0      1/10/2022 0:00           0     1    2_PREDICTION
1      1/10/2022 0:00           0     5    2_PREDICTION
2      1/10/2022 0:00           0     9    2_PREDICTION
3      1/10/2022 0:30           0     1    2_PREDICTION
4      1/10/2022 0:30           0     5    2_PREDICTION
...               ...         ...   ...             ...
2299  1/25/2022 23:00           0     5    2_PREDICTION
2300  1/25/2022 23:00           0     9    2_PREDICTION
2301  1/25/2022 23:30           0     1    2_PREDICTION
2302  1/25/2022 23:30           0     5    2_PREDICTION
2303  1/25/2022 23:30           0     9    2_PREDICTION

[2304 rows x 4 columns]


In [6]:
#Append TRAIN and PREDICT data set
#This joint is made to reduce importing code, but it is clear that TRAIN and PREDICT should be treated by appart

df_input = df_input_01.append(df_input_02).copy()
df_input = df_input.reset_index(drop=True)
print(df_input.shape)

print(df_input['set_description'].value_counts(dropna=False).reset_index().sort_values(['index'],ascending = True))


(54147, 5)
          index  set_description
0       1_TRAIN            51843
1  2_PREDICTION             2304


In [7]:
df_input.columns.tolist()

['date', 'q_current', 'Store', 'set_description', 'q_predicted']

In [8]:
df_raw = df_input.copy()
print(df_raw.shape)

(54147, 5)


# Adjusting the formats of the data

In [9]:
#ADJUSTS VARIABLE NAMES

df_input = df_raw.copy()

D_COLS_01 = df_input.dtypes.reset_index()['index']

D_COLS =[]
D_VARS = {}
for x in D_COLS_01:
    D_NAME = x.strip().lower()
    D_NAME = D_NAME.replace(" ", '_')
    D_NAME = D_NAME.replace(".", '')
    D_NAME = D_NAME.replace("�", '')
    D_NAME = D_NAME.replace("-", '_')
    D_NAME = D_NAME.replace("/", '_')
    
    ls_dupnames = [x for x in D_COLS if D_NAME in x]
    if len(ls_dupnames)>0:
        D_NAME = D_NAME + '_' + str(len(ls_dupnames))
        
    
    d = {D_NAME:str}
    D_VARS.update(d)
    
    D_COLS.append(D_NAME)


df_input.columns = D_VARS

df_raw_01 = df_input.copy()

i = 0
for c in df_raw_01.columns.tolist():
    i = i +1
    print(i, '-', c)

1 - date
2 - q_current
3 - store
4 - set_description
5 - q_predicted


In [10]:
df_informats = df_raw_01.dtypes.reset_index()
df_informats = df_informats[df_informats[0]!='object']

df_informats

,index,0


In [11]:
# Defines the main formats of the variables


#Lists numerical variables
LS_NUMVAR = [
    'q_current',
    'store',
    'q_predicted',
]

#List of categorical variables
LS_CATVAR = [
    'date',
    'set_description',
]

In [12]:
# TRANSFORMS CATEGORICAL VARIABLES

df_input = df_raw_01.copy() 

def convertscat(x):
    try:
        y = round(x) #IF THE CATEGORY LOOK LIKE A NUMBER, BETTER ROUND IT
        y = str(y).strip() 
    except:
        try:
            y = str(x).upper().strip() #THE CATEGORY IS TRANSFORMED ONTO UPCASE AND REMOVED THE BLANK TAILS
        except:
            y = np.nan #PUT A NAN IN CASE OF NOT CONVERSION
    return y


for s in LS_CATVAR:
    CATVAR = df_input[s].apply(lambda x: convertscat(x))
    df_input = df_input.drop(columns=[s])
    df_input[s] = CATVAR.astype(str)
    
    print('Variable transformed: ' + str(s))

df_raw_02 = df_input.copy()
del(df_input)

print('\n')
print(df_raw_02.shape)
print('\n')
print(df_raw_02.dtypes)
print('\n')
df_raw_02[LS_CATVAR].head(10)

Variable transformed: date
Variable transformed: set_description


(54147, 5)


q_current          object
store              object
q_predicted        object
date               object
set_description    object
dtype: object




,date,set_description
0,1/1/2021 0:00,1_TRAIN
1,1/1/2021 0:00,1_TRAIN
2,1/1/2021 0:00,1_TRAIN
3,1/1/2021 0:30,1_TRAIN
4,1/1/2021 0:30,1_TRAIN
5,1/1/2021 0:30,1_TRAIN
6,1/1/2021 1:00,1_TRAIN
7,1/1/2021 1:00,1_TRAIN
8,1/1/2021 1:00,1_TRAIN
9,1/1/2021 1:30,1_TRAIN


In [13]:
#TRANSFORMS NUMERICAL VARIABLES

df_input = df_raw_02.copy() 

def converts(x):
    try:
        y = str(x)                  #FIRST THE VARIABLES IS LOOK AS STRING
        y = y.replace("$", '')      #REMOVES THE $ SIMBOL
        y = y.replace(",", '.')     #Replace the , for . (depends of the numeric format of origin)
        #y = y.replace(",", '')     #Replace the , for '' (depends of the numeric format of origin)
        y = float(y)                # Converts to float
    except:
        y = np.nan
    return y


for s in LS_NUMVAR:
    NUMVAR = df_input[s].apply(lambda x: converts(x))
    df_input = df_input.drop(columns=[s])
    df_input[s] = NUMVAR.astype(float)
    
    
    
    print('Variable transformed: ' + str(s))

df_raw_03 = df_input.copy()
del(df_input)

print('\n')
print(df_raw_03.shape)
print('\n')
print(df_raw_03.dtypes)
print('\n')
df_raw_03[LS_NUMVAR].head(10)

Variable transformed: q_current
Variable transformed: store
Variable transformed: q_predicted


(54147, 5)


date                object
set_description     object
q_current          float64
store              float64
q_predicted        float64
dtype: object




,q_current,store,q_predicted
0,87.0,1.0,NaN
1,145.0,5.0,NaN
2,49.0,9.0,NaN
3,102.0,1.0,NaN
4,170.0,5.0,NaN
5,43.0,9.0,NaN
6,120.0,1.0,NaN
7,165.0,5.0,NaN
8,45.0,9.0,NaN
9,121.0,1.0,NaN


In [14]:
#VERIFIES THE NUMERICAL CONVERSION 

DF_INPUT = df_raw_03.copy()
DF_INPUT2 = df_raw_02.copy()

for c in LS_NUMVAR:
    print(DF_INPUT[c].value_counts(dropna=False).reset_index().sort_values([c],ascending = False))
    print(DF_INPUT2[c].value_counts(dropna=False).reset_index().sort_values([c],ascending = False))

     index  q_current
0      NaN       2304
1      3.0       1740
2      2.0       1724
3      4.0       1656
4      5.0       1358
..     ...        ...
251  254.0          1
252  263.0          1
253  242.0          1
254  255.0          1
255  258.0          1

[256 rows x 2 columns]
    index  q_current
0     NaN       2304
1       3       1740
2       2       1724
3       4       1656
4       5       1358
..    ...        ...
251   254          1
252   263          1
253   242          1
254   255          1
255   258          1

[256 rows x 2 columns]
   index  store
0    1.0  18049
1    5.0  18049
2    9.0  18049
  index  store
0     1  18049
1     5  18049
2     9  18049
   index  q_predicted
0    NaN        51843
1    0.0         2304
  index  q_predicted
0   NaN        51843
1     0         2304


In [15]:
# Stablishes Date variables

D_DATEVARS = [
    'date',
]

df_raw_03[D_DATEVARS]

,date
0,1/1/2021 0:00
1,1/1/2021 0:00
2,1/1/2021 0:00
3,1/1/2021 0:30
4,1/1/2021 0:30
...,...
54142,1/25/2022 23:00
54143,1/25/2022 23:00
54144,1/25/2022 23:30
54145,1/25/2022 23:30


In [16]:
#FIXING DATE VARIABLES

df_input = df_raw_03.copy()


def validdate(x,v):
    
    try:
        x1 = x[v].split('/')
    except:
        x1 = None
        
    y1 = None
        
    try:
        year = int(x1[2][0:4])
        month = int(x1[0])
        day = int(x1[1])
        
        y1 = datetime.date(year ,month , day)
        
    except:
        y1 = None

    return y1


for v in D_DATEVARS:
    
    DF_DATE = df_input.apply(lambda x: validdate(x,v),axis=1)
    DF_DATE = pd.DataFrame(pd.to_datetime(DF_DATE, errors='coerce'))
    
    DS_INPUT = df_input.drop(columns=[v])
    df_input[v] = DF_DATE
    
    print('Variable transformed: ' + str(v)) 

    
df_raw_05 = df_input.copy()
del(df_input)

Variable transformed: date


In [17]:
df_raw_05[D_DATEVARS]

,date
0,2021-01-01
1,2021-01-01
2,2021-01-01
3,2021-01-01
4,2021-01-01
...,...
54142,2022-01-25
54143,2022-01-25
54144,2022-01-25
54145,2022-01-25


In [19]:
print(df_raw_05['date'].value_counts(dropna=False).reset_index().sort_values(['index'],ascending = False))

         index  date
375 2022-01-25   144
121 2022-01-24   144
122 2022-01-23   144
123 2022-01-22   144
124 2022-01-21   144
..         ...   ...
255 2021-01-05   144
256 2021-01-04   144
257 2021-01-03   144
24  2021-01-02   144
0   2021-01-01   144

[377 rows x 2 columns]


In [20]:
#REPORTS THE FINAL FORMAT FOR THE VARIABLES
df_input = df_raw_05.copy()

DF_FCOLUMNS = df_input.dtypes.reset_index()
DF_FCOLUMNS = DF_FCOLUMNS.rename(columns={'index':'INPUT_VAR',0:'MANDATORY_FORMAT'})

#Final number of numerical and categorical variables

Q_NUMVARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('float|int')])
Q_CATARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('float|int')==False])
Q_DATEARS = len(DF_FCOLUMNS[DF_FCOLUMNS['MANDATORY_FORMAT'].astype(str).str.contains('date')])

print('Number of Suggested Numerical Variables: ' + str(Q_NUMVARS))   
print('Number of Suggested Categorical Variables: ' + str(Q_CATARS))
print('Number of Final Date Variables: ' + str(Q_DATEARS)) 

Number of Suggested Numerical Variables: 3
Number of Suggested Categorical Variables: 2
Number of Final Date Variables: 1


In [21]:
# Data sumary

print(df_raw_05.shape)
print('\n')

i = 0
for c in df_raw_05.columns.tolist():
    i = i +1
    print(i, '-', c)

print('\n')
print(df_raw_05.dtypes)


(54147, 5)


1 - date
2 - set_description
3 - q_current
4 - store
5 - q_predicted


date               datetime64[ns]
set_description            object
q_current                 float64
store                     float64
q_predicted               float64
dtype: object


In [22]:
# Saving Point

df_raw_05.to_pickle(OUTPUT_DIR + 'df_temporalsales_s1.pkl')